#  tweetshashtag2vec

In [25]:
#import tensorflow backend
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import keras
from keras.layers import *
from keras.models import Sequential, Model
from keras.optimizers import *
from keras.preprocessing import sequence
print(keras.__version__)
print(keras.backend.backend())
import numpy as np
from gensim.models.doc2vec import TaggedDocument
import gensim
import multiprocessing
import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13461793467362767801
]
2.1.2
tensorflow


In [3]:
#load data
with open("temp/neg_sample_data.json", "r", encoding="utf-8") as f:
    md_nfreq_sample = json.load(f)
    
with open("temp/tweets4classification.json", "r", encoding="utf-8") as f:
    md_freq_sample = json.load(f)

In [3]:
md_nfreq_sample['data'][0]

{'hashtag_label': [],
 'hashtags': ['cervicalhealthawarenessmonth', 'obamacar'],
 'id': '418769437261455360',
 'orignal_hashtags': ['#cervicalhealthawarenessmonth', '#obamacare'],
 'raw': 'rt @ppsne: jan. is #cervicalhealthawarenessmonth. thanks to #obamacare, pap tests + the hpv vaccine are covered without a copay. http://t.c…',
 'text': 'rt : jan. is cervicalhealthawarenessmonth. thanks to obamacare, pap tests + the hpv vaccine are covered without a copay.',
 'words': ['rt',
  ':',
  'jan.',
  'is',
  'cervicalhealthawarenessmonth',
  '.',
  'thanks',
  'to',
  'obamacare',
  ',',
  'pap',
  'tests',
  '+',
  'the',
  'hpv',
  'vaccine',
  'are',
  'covered',
  'without',
  'a',
  'copay',
  '.']}

In [4]:
md_freq_sample['data'][0]

{'hashtag_label': [1, 5],
 'hashtags': ['hpv', 'vaccin'],
 'id': '418263863772327936',
 'orignal_hashtags': ['#hpv', '#vaccine'],
 'raw': 'rt @cdcstd: #hpv vax coverage could be 93% if doctors gave hpv #vaccine each time a preteen/teen got any other vaccine&gt; http://t.co/xxryga5…',
 'text': 'rt : hpv vax coverage could be 93% if doctors gave hpv vaccine each time a preteen / teen got any other vaccine>',
 'words': ['rt',
  ':',
  'hpv',
  'vax',
  'coverage',
  'could',
  'be',
  '93',
  '%',
  'if',
  'doctors',
  'gave',
  'hpv',
  'vaccine',
  'each',
  'time',
  'a',
  'preteen',
  '/',
  'teen',
  'got',
  'any',
  'other',
  'vaccine',
  '>']}

# gensim implementation of tweetshashtag2vec using doc2vec api

In [5]:
import gensim

E:\conda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## train a doc2vec based on a dataset with all hashtags

In [6]:
#data prepare
train_raw_data = md_freq_sample['data'] +  md_nfreq_sample['data']
print(train_raw_data[0]['words'])
print(len(train_raw_data))
train_raw_data[0]

['rt', ':', 'hpv', 'vax', 'coverage', 'could', 'be', '93', '%', 'if', 'doctors', 'gave', 'hpv', 'vaccine', 'each', 'time', 'a', 'preteen', '/', 'teen', 'got', 'any', 'other', 'vaccine', '>']
115165


{'hashtag_label': [1, 5],
 'hashtags': ['hpv', 'vaccin'],
 'id': '418263863772327936',
 'orignal_hashtags': ['#hpv', '#vaccine'],
 'raw': 'rt @cdcstd: #hpv vax coverage could be 93% if doctors gave hpv #vaccine each time a preteen/teen got any other vaccine&gt; http://t.co/xxryga5…',
 'text': 'rt : hpv vax coverage could be 93% if doctors gave hpv vaccine each time a preteen / teen got any other vaccine>',
 'words': ['rt',
  ':',
  'hpv',
  'vax',
  'coverage',
  'could',
  'be',
  '93',
  '%',
  'if',
  'doctors',
  'gave',
  'hpv',
  'vaccine',
  'each',
  'time',
  'a',
  'preteen',
  '/',
  'teen',
  'got',
  'any',
  'other',
  'vaccine',
  '>']}

In [24]:
docs = [TaggedDocument(each['words'], each['orignal_hashtags']) for each in train_raw_data]
words_count = sum(len(each[0]) for each in docs)
tags_count = sum(len(each[1]) for each in docs)
print(words_count)
print(tags_count)
np.random.shuffle(docs)
docs[0]

5995513
239984


TaggedDocument(words=['rt', ':', 'the', 'cdc', 'hpv', 'portal', 'has', 'the', 'information', 'you', 'need', 'about', 'cancer', ',', 'vaccine', ',', '&', 'std', 'aspects', 'of', 'hpv', ';'], tags=['#hpv', '#cancer', '#vaccine', '#std'])

In [102]:
%%time 
#training model manully tune alpha
#using PV-DM w/ concatenation - window=5 (both sides) 
model = gensim.models.doc2vec.Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, alpha=0.025, min_alpha=0.025, workers=multiprocessing.cpu_count(), iter = 5)
model.build_vocab(docs)

for epoch in range(100):
    if epoch % 10 == 0:
        print ('Now training epoch %s'%epoch)
    model.train(docs, total_examples = words_count, epochs = model.iter)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

Now training epoch 0
Now training epoch 10
Now training epoch 20
Now training epoch 30
Now training epoch 40
Now training epoch 50
Now training epoch 60
Now training epoch 70
Now training epoch 80
Now training epoch 90
Wall time: 21min


In [100]:
# shows the similar words
print (model.wv.most_similar('vaccine'))
print() 
# shows the learnt embedding
print (model.wv['cancer'])
print() 
# shows the similar docs with tag = #hpv
print (model.docvecs.most_similar("#vaccine"))
print (model.docvecs.most_similar("#vaccination"))
print (model.docvecs.most_similar("#vaccines"))
print() 
#hashtag embeddings
print(model.docvecs['#cervical'])

[('vaccination', 0.6498515605926514), ('vacine', 0.6334244608879089), ('vax', 0.6272422075271606), ('vaccines', 0.5875192284584045), ('musical', 0.5415578484535217), ('vaccinations', 0.5082309246063232), ('worstwaystostartaloveletter', 0.5036842226982117), ('jabs', 0.5027507543563843), ('vaccine-', 0.48801249265670776), ('virus', 0.4854571223258972)]

[ 0.00742153 -0.12589392  0.09254588 -0.53934532 -0.37409088 -0.49017796
 -0.48392534  0.30872864  0.07505405 -0.39583573 -0.01200234  0.01792545
  0.47690091 -0.08285615 -0.07408908  0.41711783  0.4188571  -0.25587532
 -0.57685226  0.21157433 -0.19709808  0.23705886  0.4958131  -0.39376715
  0.01145776  0.20706977 -0.08154003 -0.05301575  0.41186711 -0.21266128
  0.3451131   0.11038587  0.15867655 -0.33747068  0.51296878  0.46035737
  0.10456064  0.21894766  0.14575906 -0.09127235 -0.34013075 -0.38483164
 -0.19459721 -0.68410045 -0.58127201 -0.6700176  -0.2505987   0.46651417
  0.59480202  0.11337936 -0.36305392  0.01898085  0.86931616 -

In [87]:
#assess model
origin = model.docvecs['#hpv'] 
word_sims = [('word', word, score) for word, score in model.most_similar([origin],topn=20)]
tag_sims = [('tag', tag, score) for tag, score in model.docvecs.most_similar([origin],topn=20)]
results = sorted((tag_sims + word_sims),key=lambda tup: -tup[2])
results[:20]

E:\conda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('tag', '#nocure', 13.237060546875),
 ('tag', '#arhp', 13.116081237792969),
 ('tag', '#actslikecancer', 13.039281845092773),
 ('word', 'ssmear', 12.815620422363281),
 ('word', 'agenda-driven', 12.494791984558105),
 ('word', 'donotmakeexcuses', 11.706401824951172),
 ('tag', '#canturntocancer', 11.06768798828125),
 ('word', '122,844', 10.25916862487793),
 ('word', 'hpvfirst', 10.030200004577637),
 ('word', 'thinkcurefightscancer', 9.484886169433594),
 ('word', 'gymsm', 9.359027862548828),
 ('word', 'billgatesfoundation', 9.23710823059082),
 ('word', 'hinduwomen', 8.3189697265625),
 ('tag', '#usnews', 7.730393886566162),
 ('word', 'gene-editing', 7.472973823547363),
 ('word', 'rar', 7.298567771911621),
 ('word', 'repostwhiz', 7.176763534545898),
 ('tag', '#500pxrtg', 7.152878761291504),
 ('word', 'ahem', 7.11164665222168),
 ('word', "'india", 5.6226806640625)]

In [88]:
#another evaluation
test_set = np.random.choice(train_raw_data, int(len(train_raw_data)*0.2))
test_tag = [each['orignal_hashtags'] for each in test_set]
test_seq = [each['words'] for each in test_set]
print(test_tag[:2])
print(test_seq[:2])

[['#acip', '#hpv', '#vaccine', '#vaccin'], ['#gardasil', '#vaccine']]
[['rt', ':', 'good', 'news', 'for', 'everyone', ':', 'cdc', 'acip', 'approved', 'a', '2-dose', 'schedule', ',', '6', 'months', 'apart', ',', 'for', 'hpv', 'vaccine', 'for', '9-14', 'year', 'olds', '.', 'vaccin'], ['rt', ':', 'dr.', 'harper', ':', 'gardasil', 'was', 'fast-tracked', 'vaccine', ',', 'so', 'we', 'do', "n't", 'know', 'long-term', 'side', 'effects']]


In [89]:
print(test_set[0]['text'])
inferred_vec = model.infer_vector(test_set[0]['words'])
print(test_set[0]['orignal_hashtags'])
model.docvecs.most_similar([inferred_vec])

rt : good news for everyone: cdc acip approved a 2-dose schedule, 6 months apart, for hpv vaccine for 9-14 year olds. vaccin
['#acip', '#hpv', '#vaccine', '#vaccin']


[('#actslikecancer', 18.694869995117188),
 ('#nocure', 16.895462036132812),
 ('#arhp', 16.645540237426758),
 ('#canturntocancer', 16.18458366394043),
 ('#usnews', 10.804193496704102),
 ('#500pxrtg', 8.893560409545898),
 ('#terrible', 0.9951874017715454),
 ('#acip', 0.8571861386299133),
 ('#sucks', 0.8467624187469482),
 ('#therebelpatient', 0.8244351148605347)]

In [90]:
for i, each in enumerate(test_set):
    if i < 10:
        inferred_vec = model.infer_vector(each['words'])
        print("tweet >> \n {}".format(each['text']))
        print("hashtags of tweet >>\n {}".format(each['orignal_hashtags']))
        inferred_res = model.docvecs.most_similar([inferred_vec], topn=5)
        print("predicated hashtags based on tweet >>\n {}".format(inferred_res))
        print("the top 5 second level infer hashtags >>\n {}".format(model.docvecs.most_similar(inferred_res[0][0], topn=5)))
        print()

tweet >> 
 rt : good news for everyone: cdc acip approved a 2-dose schedule, 6 months apart, for hpv vaccine for 9-14 year olds. vaccin
hashtags of tweet >>
 ['#acip', '#hpv', '#vaccine', '#vaccin']
predicated hashtags based on tweet >>
 [('#actslikecancer', 18.50225257873535), ('#nocure', 16.6639347076416), ('#arhp', 16.425540924072266), ('#canturntocancer', 15.531455993652344), ('#usnews', 10.321876525878906)]
the top 5 second level infer hashtags >>
 [('#nocure', 27.818897247314453), ('#canturntocancer', 26.703472137451172), ('#arhp', 24.8717041015625), ('#usnews', 15.32882022857666), ('#500pxrtg', 13.759662628173828)]

tweet >> 
 rt : dr. harper: gardasil was fast-tracked vaccine, so we don't know long-term side effects
hashtags of tweet >>
 ['#gardasil', '#vaccine']
predicated hashtags based on tweet >>
 [('#actslikecancer', 19.759796142578125), ('#nocure', 19.584598541259766), ('#arhp', 17.96923065185547), ('#canturntocancer', 17.141000747680664), ('#usnews', 9.855619430541992)]


## train a t2v model with a dataset containing only tweets with top 11 freq hashtags and tid

In [119]:
docs_1 = [TaggedDocument(each['words'], each['hashtags']) for each in md_freq_sample['data']]
np.random.shuffle(docs_1)
docs_1[0]

TaggedDocument(words=['vaccineswork', 'the', 'hpv', 'vaccine', 'it', "'s", 'cancer', 'prevention', 'in', 'the', 'hands', 'of', 'physicians', 'and', 'parents'], tags=['vaccineswork', '846524352418004992'])

In [121]:
%%time
#using skip-gram model
model_1 = gensim.models.doc2vec.Doc2Vec(dm=0, dbow_words=1, size=100, alpha=0.025, min_alpha=0.025, min_count=0, workers=multiprocessing.cpu_count(), iter = 10)
model_1.build_vocab(docs_1)

for epoch in range(100):
    if epoch % 10 == 0:
        print ('Now training epoch %s'%epoch)
    model_1.train(docs, total_examples = words_count, epochs = model.iter)
    model_1.alpha -= 0.002  # decrease the learning rate
    model_1.min_alpha = model.alpha  # fix the learning rate, no decay

Now training epoch 0
Now training epoch 10
Now training epoch 20
Now training epoch 30
Now training epoch 40
Now training epoch 50
Now training epoch 60
Now training epoch 70
Now training epoch 80
Now training epoch 90
Wall time: 55min 45s


In [152]:
# shows the similar words
print (model_1.wv.most_similar('vaccine'))
print() 
# shows the learnt embedding
print (model_1.wv['cancer'])
print() 
# print (model_1.docvecs.most_similar("vaccine"))
# print (model_1.docvecs.most_similar("vaccination"))
# print (model_1.docvecs.most_similar("vaccines"))
print() 
#hashtag embeddings
print(model_1.docvecs['hpv'])

[('hpv', 0.8945080041885376), ('!', 0.8925163745880127), ('10', 0.8894655108451843), ('report', 0.8864502906799316), ('out', 0.8853661417961121), (',', 0.8850535154342651), ('of', 0.8846757411956787), ('with', 0.8832195401191711), ('from', 0.8825470805168152), ('who', 0.8825079202651978)]

[-2.33820486  0.48245114  0.14271986 -0.61947644  0.14949125 -0.75954616
 -0.75303531 -1.0440402  -0.69968951 -0.46689004 -1.11501551  1.84001029
 -1.32465363  0.42597178  2.15874457 -1.02550423  0.02361116 -1.13783693
 -1.76253951 -2.7321949   2.33409142  0.88917965  0.92387992 -0.41094562
 -0.4912841  -1.95612872  1.51078582  1.41663611 -0.56113583  0.906766
 -1.88180816  0.77339727  0.09722759  1.64448333  0.2642459   0.74397922
  0.45687717 -1.05445135 -1.63638735 -1.28904271  2.27400208 -1.37547159
  0.34097117 -0.81782961 -0.44842571 -1.06702149 -0.29075247  1.80298615
 -1.23309886 -1.62950492 -0.21880794 -0.45533019 -2.01157761  0.80548465
 -0.2964218   1.31826317  0.06735939  0.41151595 -2.47

TypeError: '<' not supported between instances of 'str' and 'int'

In [123]:
test_set = np.random.choice(train_raw_data, int(len(md_freq_sample['data'])*0.2))

for i, each in enumerate(test_set):
    if i < 10:
        inferred_vec = model_1.infer_vector(each['words'])
        print("tweet >> \n {}".format(each['text']))
        print("hashtags of tweet >>\n {}".format(each['hashtags']))
        inferred_res = model_1.docvecs.most_similar([inferred_vec], topn=5)
        print("predicated hashtags based on tweet >>\n {}".format(inferred_res))
        print("the top 5 second level infer hashtags >>\n {}".format(model_1.docvecs.most_similar(inferred_res[0][0], topn=5)))
        print()

tweet >> 
 rt : mt : hpv vaccination is cancer prevention. be part of the solution. gyncsm
hashtags of tweet >>
 ['hpv', 'vaccin', 'cancer', 'gyncsm', '657569260365942784']
predicated hashtags based on tweet >>
 [('chile', 0.47245335578918457), ('628044565233037312', 0.4166824221611023), ('767802779717005312', 0.3938097059726715), ('883171981306277888', 0.3748818635940552), ('507164992861659136', 0.3747347593307495)]
the top 5 second level infer hashtags >>
 [('906304552365953026', 0.41449832916259766), ('849624851627343872', 0.407646119594574), ('687684481612967940', 0.40286827087402344), ('907980338475003904', 0.40044668316841125), ('909984795047923712', 0.4002317786216736)]

tweet >> 
 free hpv vaccine provider education course available online! earn credit and learn tips
hashtags of tweet >>
 ['hpv', '656548161356451840']
predicated hashtags based on tweet >>
 [('558611813399400448', 0.4195649325847626), ('726168748017762306', 0.39725983142852783), ('741099343931973632', 0.39422643

## another model

In [103]:
%%time
#training model
model2 = gensim.models.doc2vec.Doc2Vec(dm=0, dbow_words=1, size=100, alpha=0.025, min_alpha=0.025, min_count=1, workers=multiprocessing.cpu_count(), iter=100, window = 30)
model2.build_vocab(docs)
model2.train(docs, total_examples = words_count, epochs = model.iter)

Wall time: 1min 6s


In [104]:
test_set = np.random.choice(train_raw_data, int(len(md_freq_sample['data'])*0.2))

for i, each in enumerate(test_set):
    if i < 10:
        inferred_vec = model2.infer_vector(each['words'])
        print("tweet >> \n {}".format(each['text']))
        print("hashtags of tweet >>\n {}".format(each['hashtags']))
        inferred_res = model2.docvecs.most_similar([inferred_vec], topn=5)
        print("predicated hashtags based on tweet >>\n {}".format(inferred_res))
        print("the top 5 second level infer hashtags >>\n {}".format(model2.docvecs.most_similar(inferred_res[0][0], topn=5)))
        print()

tweet >> 
 nothing is more exciting than new indications! keytruda , taltz, & gardasil
hashtags of tweet >>
 ['keytruda', 'taltz', 'gardasil']
predicated hashtags based on tweet >>
 [('#keytruda', 0.8704658150672913), ('#taltz', 0.8633818626403809), ('#pesticide', 0.7911003232002258), ('#cooncology', 0.7874743938446045), ('#ozone', 0.7840933799743652)]
the top 5 second level infer hashtags >>
 [('#taltz', 0.96110999584198), ('#timelapse', 0.8220064640045166), ('#production', 0.8104546070098877), ('#cooncology', 0.8099256157875061), ('#denveroncology', 0.8096989989280701)]

tweet >> 
 we have a vaccine that can eliminate up to 30,000 cancers a year. yet hpv vaccination rates remain low
hashtags of tweet >>
 ['hpv']
predicated hashtags based on tweet >>
 [('#nursepractitioners', 0.7152985334396362), ('#pnp', 0.7023173570632935), ('#fnp', 0.696955144405365), ('#irvinehome', 0.6947712898254395), ('#kansashospitals', 0.6944798231124878)]
the top 5 second level infer hashtags >>
 [('#pnp', 0

In [160]:
#only with id as tag model to varify the model is validate to use
docs_2 = [TaggedDocument(each['words'], [each['id']]) for each in train_raw_data]
np.random.shuffle(docs_2)
docs_2[0]

TaggedDocument(words=['rt', ':', 'a3', 'woc', 'are', 'disproportionately', 'impacted', 'by', 'hiv', '&', 'breast', '+', 'cervical', 'cancer', '.', 'high', 'costs', 'of', 'ahca', 'means', 'many', 'will', 'go', 'w', '/', 'o', 'care', 'pr'], tags=['870025156185407488'])

In [161]:
%%time
#train model
model3 = gensim.models.doc2vec.Doc2Vec(dm=0, dbow_words=1, size=100, alpha=0.025, min_alpha=0.025, min_count=1, workers=multiprocessing.cpu_count(), iter=100)
model3.build_vocab(docs_2)
for epoch in range(100):
    if epoch % 10 == 0:
        print ('Now training epoch %s'%epoch)
    model3.train(docs_2, total_examples = words_count, epochs = model.iter)
    model3.alpha -= 0.002  # decrease the learning rate
    model3.min_alpha = model.alpha  # fix the learning rate, no decay

Now training epoch 0
Now training epoch 10
Now training epoch 20
Now training epoch 30
Now training epoch 40
Now training epoch 50
Now training epoch 60
Now training epoch 70
Now training epoch 80
Now training epoch 90
Wall time: 1h 15min 38s


In [162]:
#evaluation
test_set = np.random.choice(train_raw_data, 5)
for each in test_set:
    inferred_vec = model3.infer_vector(each['words'])
    print("tweet >> \n {}".format(each['text']))
    print("hashtags of tweet >>\n {}".format(each['hashtags']))
    inferred_res = model3.docvecs.most_similar([inferred_vec], topn=3)
    print("predicated id based on tweet >>\n {}".format(inferred_res))
    for each in inferred_res:
        sim = find_tweet_by_id(each[0], train_raw_data)
        print("similar tweets is >> \n{}".format(sim['text']))
        print("similar tweets hashtags are >> \n{}".format(sim['hashtags']))
    #print("the top 3 second level inferred id >>\n {}".format(model3.docvecs.most_similar(inferred_res[0][0], topn=3)))
    print()

tweet >> 
 . chooses ideology over recommendations from medical professionals on hpv letter ingov
hashtags of tweet >>
 ['ingov']
predicated id based on tweet >>
 [('808100784625700864', 0.8606292605400085), ('844142397248475137', 0.834769606590271), ('582318075258613760', 0.8021253347396851)]
similar tweets is >> 
gardasil vaccine safety  under attack again by a false authority - cancer hpv vaccineswork
similar tweets hashtags are >> 
['gardasil', 'cancer', 'hpv', 'vaccineswork', '808100784625700864']
similar tweets is >> 
rt : olympic medalist dies of cervical cancer after hpv vaccine  gardasil learntherisk
similar tweets hashtags are >> 
['cancer', 'hpv', 'vaccin', 'gardasil', 'learntherisk', '844142397248475137']
similar tweets is >> 
rt : gardasil 9 vaccine protects against additional types of hpv: cervicalca hncsm cancer
similar tweets hashtags are >> 
['hpv', 'cervicalca', 'hncsm', 'cancer', '582318075258613760']

tweet >> 
 rt : henriettalacks' cells are still helping protect w

In [8]:
def find_tweet_by_id(tid, tweets):
    for each in tweets:
        if each['id'] == tid:
            return each
    return null

In [9]:
#another eval
tweet_id = np.random.choice(train_raw_data, 1)[0]['id']
print(tweet_id)

tweet = find_tweet_by_id(tweet_id, train_raw_data)
print("picked tweet >>\n {}".format(tweet['text']))

sims = model3.docvecs.most_similar([tweet_id])

#print(find_tweet_by_id(sims[0][0], train_raw_data))

for l, i in [('MOST', 0), ("Median", len(sims)//2), ('LEAST', len(sims)-1)]:
    print("{}\t{} >>\n{}".format(l, i, find_tweet_by_id(sims[i][0], train_raw_data)['text']))

901122406546362368
picked tweet >>
 rt : preteens can get hpvvaccine when they receive other preteen vaccines. see 6 reasons to get hpv vax for your child:


NameError: name 'model3' is not defined

## model with all tweets merged based on same hashtag

In [10]:
%%time
#prepare training data
uniq_hashtags = dict()
for each in train_raw_data:
    for tag in each['orignal_hashtags']:
        if tag not in uniq_hashtags:
            uniq_hashtags[tag] = each['words']
        else:
            uniq_hashtags[tag].append(" ")
            uniq_hashtags[tag].extend(each['words'])

Wall time: 263 ms


In [17]:
t2v_data = (md_nfreq_sample['data'] + md_freq_sample['data'])

In [27]:
docs4 = []
for each in t2v_data:
    if len(each['hashtags']) != 0:
        docs4.append(TaggedDocument(each['words'], ([each['id']] + each['hashtags'])))
np.random.shuffle(docs4)
words_count = sum(len(each[0]) for each in docs4)
tags_count = sum(len(each[1]) for each in docs4)
docs4[:5]

[TaggedDocument(words=['sessionspromotinghealth', 'set', 'cervicalcancer'], tags=['911055768484499457', 'cervicalcanc']),
 TaggedDocument(words=['the', 'latest', 'the', 'cervical', 'cancer', ',', 'toxic', 'shock', 'syndrome', ',', 'hpv', ',', 'sanitary', 'pads', ',', 'women', "'s", 'healthcare', 'daily', '!', 'comeclean'], tags=['885541974761144320', 'comeclean']),
 TaggedDocument(words=['.', 'too', 'many', 'girls', 'are', 'getting', 'injured', 'or', 'dying', 'by', 'hpv', 'vaccine', '.', 'factsonvax'], tags=['715568151748956160', 'factsonvax']),
 TaggedDocument(words=['eval', 'of', 'abbott', 'realtime', 'hr', 'hpv', 'test', 'vs', 'hc2', 'hr', 'hpv', 'test', 'in', 'women', '30', 'yrs', 'using', 'residual', 'lbc', 'thinprep', 'specimens'], tags=['805388765342691328', 'hpv']),
 TaggedDocument(words=['``', 'hpv', 'vaccine', 'could', 'protect', 'more', 'people', 'with', 'fewer', 'doses', ',', 'doctors', 'insist', "''", 'on', ':', 'health', 'science', 'npr'], tags=['847093788459896832', 'hea

In [39]:
%%time
#train model
model4 = gensim.models.doc2vec.Doc2Vec(compute_loss=True, dm=1, window=10, alpha=0.01, min_alpha=0.002,  size=100, min_count=1, workers=multiprocessing.cpu_count(), iter=10)
model4.build_vocab(docs4)
# model4.train(docs4, total_examples = words_count, epochs = model4.iter)
for epoch in range(50):
#     if epoch % 10 == 0:
#         print ('Now training epoch %s'%epoch)
    model4.train(docs4, total_examples = words_count, epochs = model4.iter)
    model4.alpha -= 0.001  # decrease the learning rate
    model4.min_alpha = model4.alpha  # fix the learning rate, no decay

Wall time: 1h 17min 57s


In [40]:
#save model4
model4.save('model/Doc2Vec_twitter_gensim_idht_50.d2v')

In [41]:
model4.docvecs.similarity('hpv', '805388765342691328')

-0.36467149878173766

In [42]:
model4.docvecs.most_similar('cancer', topn=20)

[('833382483320266752', 0.7511858344078064),
 ('908863665369001984', 0.745309591293335),
 ('866594499832795137', 0.7295096516609192),
 ('905374278748692482', 0.7256207466125488),
 ('899468905160298499', 0.7220050692558289),
 ('hpv', 0.7212355136871338),
 ('836000360439427072', 0.7125046849250793),
 ('879437856254676996', 0.7107582092285156),
 ('913177633009147905', 0.7032711505889893),
 ('833472044754989056', 0.7008352279663086),
 ('841665286214410240', 0.6970347166061401),
 ('cervic', 0.6937721371650696),
 ('914500313796751360', 0.6925458908081055),
 ('882910055967936513', 0.6881334781646729),
 ('895022189099053060', 0.687397837638855),
 ('878652082424864768', 0.686741828918457),
 ('914130849687375874', 0.6857236623764038),
 ('892876480757325829', 0.6853112578392029),
 ('846664631351431168', 0.6823394894599915),
 ('838985931373887489', 0.6791170239448547)]

## twitter id 2 hashtag

- using the cnn_hashtagcls_emb200_weightscnn_biLSTM.h5 model to predict the freq_sample, match the predicated data with its hashtags and save the results with doc_id
- extract the doc vec based on each doc_id
- using doc_vec as input data and predicated hashtags as output label

In [48]:
import pickle
def load_dict(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

dictionary = load_dict('model/dict.dd')

doc = []
doc_id = []

for each in md_nfreq_sample['data']:
    doc_id.append(each['id'])
    doc.append(each['words'])

print(doc[:2])
print(doc_id[:2])
    
seq_len = 39    
X_pred = [[dictionary.index(word) if word in dictionary else 0 for word in each] for each in doc]
print(X_pred)
X_pred = sequence.pad_sequences(X_pred, maxlen=seq_len)
X_pred[:5]

[['rt', ':', 'jan.', 'is', 'cervicalhealthawarenessmonth', '.', 'thanks', 'to', 'obamacare', ',', 'pap', 'tests', '+', 'the', 'hpv', 'vaccine', 'are', 'covered', 'without', 'a', 'copay', '.'], ['rt', ':', 'jan.', 'is', 'cervicalhealthawarenessmonth', '.', 'thanks', 'to', 'obamacare', ',', 'pap', 'tests', '+', 'hpv', 'vaccine', 'covered', 'w', '/', 'out', 'copay', '.']]
['418769437261455360', '418799634572513282']
[[2, 0, 1961, 17, 1385, 4, 184, 8, 1700, 6, 80, 162, 645, 5, 1, 3, 30, 938, 266, 14, 2013, 4], [2, 0, 1961, 17, 1385, 4, 184, 8, 1700, 6, 80, 162, 645, 1, 3, 938, 54, 40, 93, 2013, 4], [2, 0, 1961, 17, 1385, 4, 184, 8, 1700, 6, 80, 162, 645, 5, 1, 3, 30, 938, 266, 14, 2013, 4], [184, 8, 1700, 6, 80, 162, 16, 5, 1, 3, 30, 154, 938, 266, 14, 2013, 18, 978, 69], [2, 0, 1961, 17, 1385, 4, 184, 8, 1700, 6, 80, 162, 645, 5, 1, 3, 30, 938, 266, 14, 2013, 4], [2, 0, 1961, 17, 1385, 4, 184, 8, 1700, 6, 80, 162, 645, 5, 1, 3, 30, 938, 266, 14, 2013, 4], [31, 20, 247, 17, 48, 188, 3147, 

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    2,    0, 1961,   17, 1385,
           4,  184,    8, 1700,    6,   80,  162,  645,    5,    1,    3,
          30,  938,  266,   14, 2013,    4],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    2,    0, 1961,   17,
        1385,    4,  184,    8, 1700,    6,   80,  162,  645,    1,    3,
         938,   54,   40,   93, 2013,    4],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    2,    0, 1961,   17, 1385,
           4,  184,    8, 1700,    6,   80,  162,  645,    5,    1,    3,
          30,  938,  266,   14, 2013,    4],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  184,    8,
        1700,    6,   80,  162,   16,    5,    1,  

In [50]:
#using model4 results to create a model for classify tweets hashtags
model_path = 'model/cnn_hashtagcls_emb200_weightscnn_biLSTM.h5'
from keras.models import load_model
model = load_model(model_path)
pred_prob = model.predict(X_pred)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [59]:
doc_vec = []
for each in doc_id:
    doc_vec.append(model4.docvecs['805388765342691328'])
doc_vec[:2]

[array([-0.01435213, -0.02807958,  0.00574302, -0.13382487, -0.15349874,
        -0.21184944, -0.25326976, -0.02308253, -0.22044262, -0.09659161,
        -0.02601681, -0.12842865, -0.12430958,  0.06735289, -0.04441446,
        -0.08872792, -0.05373222, -0.02762013, -0.0514167 ,  0.36038172,
        -0.34108534, -0.11881448,  0.09146494, -0.27072796,  0.07834855,
         0.17834699, -0.10612268,  0.34392193,  0.1901266 , -0.0844566 ,
         0.10192393,  0.3547918 ,  0.1701296 , -0.2711349 , -0.00999706,
         0.09783691,  0.17476909,  0.23835462,  0.10185689, -0.01933094,
         0.06079472,  0.22866441,  0.09315217,  0.12563862,  0.19850954,
        -0.17628457, -0.07847248,  0.03153325, -0.35311726,  0.03788557,
         0.16634382, -0.12775768,  0.34667438,  0.38534626,  0.11466559,
         0.05435922, -0.0325979 , -0.13926049, -0.00985402, -0.33832395,
        -0.22398815, -0.00687505, -0.08236906, -0.02786132,  0.00685305,
        -0.00853606,  0.21162565,  0.28759432, -0.1

In [61]:
cw_map = {'cancer': 4,
 'cervicalcanc': 6,
 'gardasil': 2,
 'health': 0,
 'hpv': 9,
 'hpvvaccin': 3,
 'learntherisk': 8,
 'studi': 7,
 'vaccin': 5,
 'vaccineswork': 10,
 'vax': 1}

idx2ht = {v: k for k, v in cw_map.items()}

In [82]:
def get_groups(data, num_gp):
    groups = []
    for each in data:
        fmax = -1
        tag = -1
        for i, prob in enumerate(each):
            if prob > fmax:
                tag = i
                fmax = prob
        groups.append(tag)
    return groups

gps = get_groups(pred_prob, 11) 
gps[:5]

[1, 1, 1, 1, 1]

In [83]:
print(len(doc_vec))
print(len(gps))
print(len(doc_id))

34116
34116
34116


In [84]:
res = list(zip(doc_vec, gps, doc_id))
print(res[1])
np.random.shuffle(res)
cut = int(0.8*len(res))
train_set = res[:cut]
test_set = res[cut:]
test_set[:5]

(array([-0.01435213, -0.02807958,  0.00574302, -0.13382487, -0.15349874,
       -0.21184944, -0.25326976, -0.02308253, -0.22044262, -0.09659161,
       -0.02601681, -0.12842865, -0.12430958,  0.06735289, -0.04441446,
       -0.08872792, -0.05373222, -0.02762013, -0.0514167 ,  0.36038172,
       -0.34108534, -0.11881448,  0.09146494, -0.27072796,  0.07834855,
        0.17834699, -0.10612268,  0.34392193,  0.1901266 , -0.0844566 ,
        0.10192393,  0.3547918 ,  0.1701296 , -0.2711349 , -0.00999706,
        0.09783691,  0.17476909,  0.23835462,  0.10185689, -0.01933094,
        0.06079472,  0.22866441,  0.09315217,  0.12563862,  0.19850954,
       -0.17628457, -0.07847248,  0.03153325, -0.35311726,  0.03788557,
        0.16634382, -0.12775768,  0.34667438,  0.38534626,  0.11466559,
        0.05435922, -0.0325979 , -0.13926049, -0.00985402, -0.33832395,
       -0.22398815, -0.00687505, -0.08236906, -0.02786132,  0.00685305,
       -0.00853606,  0.21162565,  0.28759432, -0.17454776,  0.0

[(array([-0.01435213, -0.02807958,  0.00574302, -0.13382487, -0.15349874,
         -0.21184944, -0.25326976, -0.02308253, -0.22044262, -0.09659161,
         -0.02601681, -0.12842865, -0.12430958,  0.06735289, -0.04441446,
         -0.08872792, -0.05373222, -0.02762013, -0.0514167 ,  0.36038172,
         -0.34108534, -0.11881448,  0.09146494, -0.27072796,  0.07834855,
          0.17834699, -0.10612268,  0.34392193,  0.1901266 , -0.0844566 ,
          0.10192393,  0.3547918 ,  0.1701296 , -0.2711349 , -0.00999706,
          0.09783691,  0.17476909,  0.23835462,  0.10185689, -0.01933094,
          0.06079472,  0.22866441,  0.09315217,  0.12563862,  0.19850954,
         -0.17628457, -0.07847248,  0.03153325, -0.35311726,  0.03788557,
          0.16634382, -0.12775768,  0.34667438,  0.38534626,  0.11466559,
          0.05435922, -0.0325979 , -0.13926049, -0.00985402, -0.33832395,
         -0.22398815, -0.00687505, -0.08236906, -0.02786132,  0.00685305,
         -0.00853606,  0.21162565,  0.

In [79]:
def data_sep(data):
    d = []
    l = []
    i = []
    for each in data:
        d.append(each[0])
        l.append(each[1])
        i.append(each[2])
    return d, l, i

train_data, train_label, train_id = data_sep(train_set)
test_data, test_label, test_id = data_sep(test_set)
print(test_data[:5])
print(test_label[:5])

[array([-0.01435213, -0.02807958,  0.00574302, -0.13382487, -0.15349874,
       -0.21184944, -0.25326976, -0.02308253, -0.22044262, -0.09659161,
       -0.02601681, -0.12842865, -0.12430958,  0.06735289, -0.04441446,
       -0.08872792, -0.05373222, -0.02762013, -0.0514167 ,  0.36038172,
       -0.34108534, -0.11881448,  0.09146494, -0.27072796,  0.07834855,
        0.17834699, -0.10612268,  0.34392193,  0.1901266 , -0.0844566 ,
        0.10192393,  0.3547918 ,  0.1701296 , -0.2711349 , -0.00999706,
        0.09783691,  0.17476909,  0.23835462,  0.10185689, -0.01933094,
        0.06079472,  0.22866441,  0.09315217,  0.12563862,  0.19850954,
       -0.17628457, -0.07847248,  0.03153325, -0.35311726,  0.03788557,
        0.16634382, -0.12775768,  0.34667438,  0.38534626,  0.11466559,
        0.05435922, -0.0325979 , -0.13926049, -0.00985402, -0.33832395,
       -0.22398815, -0.00687505, -0.08236906, -0.02786132,  0.00685305,
       -0.00853606,  0.21162565,  0.28759432, -0.17454776,  0.0

In [ ]:
#encoding data

In [ ]:
#deep learning model 
model = Sequential()


In [86]:
#regression model
from sklearn import linear_model
classifier = linear_model.LogisticRegression()
classifier.fit(train_data, train_label)
classifier.score(test_data, test_label)

27292
27292


ValueError: bad input shape (27292, 11)

# t2v keras model from scratch  

In [6]:
#data
#labeled with high freq hashtag
print(md_nfreq_sample['data'][0])
#labeled with non-high freq hashtag
print(md_freq_sample['data'][0])
#no hashtag data
with open("temp/no_labeled_data.json", "r", encoding="utf-8") as f:
    md_nolabel_sample = json.load(f)
print(md_nolabel_sample['data'][0])

t2v_data = (md_nfreq_sample['data'] + md_freq_sample['data'] + md_nolabel_sample['data'])

{'id': '418769437261455360', 'text': 'rt : jan. is cervicalhealthawarenessmonth. thanks to obamacare, pap tests + the hpv vaccine are covered without a copay.', 'raw': 'rt @ppsne: jan. is #cervicalhealthawarenessmonth. thanks to #obamacare, pap tests + the hpv vaccine are covered without a copay. http://t.c…', 'orignal_hashtags': ['#cervicalhealthawarenessmonth', '#obamacare'], 'hashtags': ['cervicalhealthawarenessmonth', 'obamacar'], 'words': ['rt', ':', 'jan.', 'is', 'cervicalhealthawarenessmonth', '.', 'thanks', 'to', 'obamacare', ',', 'pap', 'tests', '+', 'the', 'hpv', 'vaccine', 'are', 'covered', 'without', 'a', 'copay', '.'], 'hashtag_label': []}
{'id': '418263863772327936', 'text': 'rt : hpv vax coverage could be 93% if doctors gave hpv vaccine each time a preteen / teen got any other vaccine>', 'raw': 'rt @cdcstd: #hpv vax coverage could be 93% if doctors gave hpv #vaccine each time a preteen/teen got any other vaccine&gt; http://t.co/xxryga5…', 'orignal_hashtags': ['#hpv', '#v

In [ ]:
#data formating for (PV-DM)CBOW
